Project 

General Steps I followed for execution of my task.

Track Connected Components Over Time: For each connected component detected in the first frame, you'll need to track its intensity or frequency changes across subsequent frames (2nd to 516th). Here's a general approach:

a. Initialize Data Structures: Create data structures to store information about each connected component detected in the first frame. This could include the component's ID, position, intensity or frequency, and any other relevant attributes.

b. Iterate Through Frames: Starting from the 2nd frame up to the 516th frame, apply the same connected component detection technique to each frame.

c. Match Connected Components: For each frame, match the detected connected components with the ones detected in the first frame. You can use techniques like centroid matching or intersection over union (IoU) to match components between frames.

d. Calculate Intensity Change or Frequency: Once components are matched between frames, calculate the intensity change or frequency of each connected component over time. Intensity change can be calculated by measuring the change in pixel intensity within the component's region of interest. Frequency change can be calculated based on how often the component appears in consecutive frames.

e. Store Data: Store the calculated intensity change or frequency for each connected component over time.

Filter Components by Frequency: After processing frames 1 to 516, filter out the connected components whose frequency falls between 0 and 1 Hz.

Repeat Process for Subsequent Frames: Repeat the above steps for the 517th to 1032nd frames.

In [1]:
import numpy as np
import cv2
from scipy.spatial import distance

Detection of components , calculating centroids

In [35]:
def detect_components(frame):
    img_grey=cv2.cvtColor( frame,cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(img_grey, 50, 255, cv2.THRESH_BINARY)# random threshold
    num_labels, labels = cv2.connectedComponents(binary_image, connectivity=8)
    
    components_image = np.zeros_like(frame, dtype=np.uint8)
    centroids = []
    for label in range(1, num_labels):  # Skip background label 0
        # Extract the connected component by filtering based on the label
        component = np.where(labels == label, 255, 0).astype(np.uint8)

        # Calculate the moments of the connected component
        moments = cv2.moments(component)

        # Calculate centroid coordinates
        if moments["m00"] != 0:
            centroid_x = int(moments["m10"] / moments["m00"])
            centroid_y = int(moments["m01"] / moments["m00"])
            centroids.append((centroid_x, centroid_y))
    
        # Calculate intensity of the connected component
        # intensity = np.sum(frame[labels == label]) / np.sum(labels == label)
    
        # Convert the component to a 3-channel image
        component = cv2.cvtColor(component, cv2.COLOR_GRAY2BGR)
    
        # Draw the connected component on the blank image
        components_image += component

        # Print intensity of the connected component
        #print(f"Intensity of component {label}: {intensity}")
        # intensity_components.append(intensity)
    return components_image,num_labels,centroids

Matching Components 

In [36]:
def centroid_matching(centroids_frame1, centroids_frame2, threshold):
    matched_component = []
    for idx1, centroid1 in enumerate(centroids_frame1):
        min_dist = float('inf')
        for idx2, centroid2 in enumerate(centroids_frame2):
            dist = distance.euclidean(centroid1, centroid2)
            if dist < min_dist:
                min_dist = dist
        if min_dist < threshold:
            matched_component.append(idx1)
    return matched_component

Calculating the frame rate

In [37]:
video_reader = cv2.VideoCapture('C:/Users/muska/Downloads/1705951007967.mp4')
# Check if the video opened successfully
if not video_reader.isOpened():
    print("Error: Could not open video.")
    exit()

# Get the frame rate of the video
frame_rate =video_reader.get(cv2.CAP_PROP_FPS)

print("Frame rate:", frame_rate)

video_reader.release()

Frame rate: 25.0


In [38]:
video_reader = cv2.VideoCapture('C:/Users/muska/Downloads/1705951007967.mp4')
number_of_frames = 516
while (True):
 ret, frame=video_reader.read()
 colored_labels_1 , Number_of_comp1 , centroid_1 = detect_components(frame)
 component_counter =[0] * Number_of_comp1
 #iterate
 i=2
 while(i<=number_of_frames):
  i=i+1
  ret, frame=video_reader.read()
  colored_labels , Number_of_comp , centroid = detect_components(frame)
  matched_components = centroid_matching(centroid_1, centroid,0.01)# random threshold
  for index, component in enumerate(matched_components):
    component_counter[component] = 1 + component_counter[component]

 frequency = []
 for i in range (0, len(component_counter)):
    frequency.append(component_counter[i] / (number_of_frames / frame_rate))

 img_grey=cv2.cvtColor( frame,cv2.COLOR_BGR2GRAY)
 _, binary_image = cv2.threshold(img_grey, 50, 255, cv2.THRESH_BINARY)# random threshold
 num_labels, labels = cv2.connectedComponents(binary_image, connectivity=8)
 components_image = np.zeros_like(frame, dtype=np.uint8)
 for f in range (1, len(frequency)):
    if (frequency[f]<=1):
        component = np.where(labels == f, 255, 0).astype(np.uint8)
         # Convert the component to a 3-channel image
        component = cv2.cvtColor(component, cv2.COLOR_GRAY2BGR)
    
        # Draw the connected component on the blank image
        components_image += component
 cv2.imshow('Video',frame)
 cv2.imshow('Video2',components_image)
 if (cv2.waitKey(1) & 0xFF == ord('q')):
    break